In [1]:
# import libraries
import os
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import collections
import gc
from sklearn.model_selection import KFold
from transformers import BertTokenizer,TFBertModel

E0000 00:00:1727139360.354901      30 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0924 00:56:00.362941255      30 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0924 00:56:00.362958023      30 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0924 00:56:00.362961282      30 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0924 00:56:00.362963644      30 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0924 00:56:00.362966221      30 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [2]:
# configure notebook to run on tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1727139396.709117      30 service.cc:145] XLA service 0x5d2cd8446360 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727139396.709163      30 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1727139396.709167      30 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1727139396.709170      30 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1727139396.709173      30 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1727139396.709176      30 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1727139396.709178      30 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1727139396.709181      30 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1727139396.709184      30 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [3]:
# load files function
text_files={}

def load_files(directory, text_files):
    for filename in os.listdir(directory):
        with open(directory+'/'+filename, encoding="utf8") as file:
            lines=''
            for line in file:
                line=line.replace("\n", "")
                lines+=line
            text_files[filename]=lines

In [4]:
# load the files, sort the files, and append files in a list
load_files('/kaggle/input/patient-text-files-zipped/patient_txt_files', text_files)
text_files = collections.OrderedDict(sorted(text_files.items()))
X=[]
for key in text_files:
    X.append(text_files[key])
X=np.array(X)

In [5]:
# load and sort the labels
y=pd.read_csv('/kaggle/input/patient-labels/Train.csv')
y.sort_values(y.columns[0], axis=0, inplace=True)
y=y['Label']
y.loc[y > 0] = 1
y=np.asarray(y).astype('float32')

In [6]:
# configure kfold split
kfold = KFold(n_splits=5, shuffle=True)

In [7]:
# define function to create, train, and evaluate model
def create_run_model(X_train, X_cv, y_train, y_cv, fold_loss, fold_accuracy, fold_precision, fold_recall, predict):
    
    # load the tokenizer and pretrained model
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    bert_model = TFBertModel.from_pretrained('bert-base-chinese')
    
    # encodes the text with tokens
    def encode_texts(texts, tokenizer, max_length=512):

        encoding = tokenizer.batch_encode_plus(texts, add_special_tokens=True,max_length=max_length,padding='max_length',truncation=True,return_tensors='tf')

        return encoding['input_ids'], encoding['attention_mask']

    # define the early stop callback function to monitor validation loss
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # define the neural network layers
    input_ids_layer = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='input_ids')
    attention_mask_layer = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='attention_mask')
    bert_output = bert_model(input_ids_layer, attention_mask=attention_mask_layer)
    dropout = tf.keras.layers.Dropout(0.12, name='dropout')(bert_output.pooler_output)
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(dropout)

    # define the model
    model = tf.keras.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=[output])

    # compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])

    # tokenize the data 
    input_ids, attention_masks=encode_texts(X_train, tokenizer)
    input_ids_cv, attention_mask_cv = encode_texts(X_cv, tokenizer)

    # train the model
    history = model.fit([input_ids, attention_masks], y_train, epochs=30, callbacks=[early_stop], validation_data=([input_ids_cv, attention_mask_cv], y_cv))

    # evaluate the model
    evaluation = model.evaluate([input_ids_cv, attention_mask_cv], y_cv)
    
    print()
    print(f"Evaluation Results: Validation Loss - {evaluation[0]}, Validation Accuracy - {evaluation[1]}, Validation Precision - {evaluation[2]}, Validation Recall - {evaluation[3]}")
    print()
    
    if predict==1:
        return 0
    
    # append evaluation metrics in lists
    else:
        fold_loss.append(evaluation[0])
        fold_accuracy.append(evaluation[1])
        fold_precision.append(evaluation[2])
        fold_recall.append(evaluation[3])

        # delete the model after training and evaluating to reset weights
        del model
        keras.backend.clear_session()

In [8]:
# initialize kfold metrics
fold = 1

fold_loss=[]
fold_accuracy=[]
fold_precision=[]
fold_recall=[]

In [9]:
# run kfold cross-validation on the training data only
with tpu_strategy.scope():

    for train, cv in kfold.split(X, y):

        X_train = X[train]
        X_cv = X[cv]
        y_train = y[train]
        y_cv = y[cv]

        print("Fold number:", fold)

        create_run_model(X_train, X_cv, y_train, y_cv, fold_loss, fold_accuracy, fold_precision, fold_recall, 0)

        fold+=1

Fold number: 1


I0000 00:00:1727139404.976191      30 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initializ

Epoch 1/30


2024-09-24 00:58:10.405312: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
I0000 00:00:1727139493.119827     849 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(d7d064156a71371a:0:0), session_name()
I0000 00:00:1727139527.456019     849 tpu_compile_op_common.cc:245] Compilation of d7d064156a71371a:0:0 with session name  took 34.336146489s and succeeded
I0000 00:00:1727139527.546673     849 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(d7d064156a71371a:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_train_function_5049897138427391008", property.function_library_fingerprint = 15840757827652016438, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, 

2/3 [===================>..........] - ETA: 0s - loss: 0.8303 - accuracy: 0.5469 - precision: 0.6667 - recall: 0.2424          

...

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/30
3/3 [==============================] - 122s 24s/step - loss: 0.6516 - accuracy: 0.5775 - precision: 0.5758 - recall: 0.9500 - val_loss: 0.6178 - val_accuracy: 0.6667 - val_precision: 0.6000 - val_recall: 0.7500
Epoch 2/30
3/3 [==============================] - 67s 19s/step - loss: 0.4609 - accuracy: 0.8592 - precision: 0.8409 - recall: 0.9250 - val_loss: 0.6042 - val_accuracy: 0.6667 - val_precision: 0.5714 - val_recall: 1.0000
Epoch 3/30
3/3 [==============================] - 66s 20s/step - loss: 0.4642 - accuracy: 0.8028 - precision: 0.7500 - recall: 0.9750 - val_loss: 0.4950 - val_accuracy: 0.7778 - val_precision: 0.6667 - val_recall: 1.0000
Epoch 4/30
3/3 [==============================] - 67s 20s/step - loss: 0.4085 - accuracy: 0.8310 - precision: 0.8333 - recall: 0.8750 - val_loss: 0.4885 - val_accuracy: 0.7778 - val_precision: 0.6667 - val_recall: 1.0000
Epoch 5/30
3/3 [==============================] - 65s 19s/step - loss: 0.3213 - accuracy: 0.8873 - precision: 0.863

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/30
3/3 [==============================] - 118s 24s/step - loss: 0.7363 - accuracy: 0.5556 - precision: 0.5517 - recall: 0.8421 - val_loss: 0.6479 - val_accuracy: 0.5882 - val_precision: 1.0000 - val_recall: 0.3000
Epoch 2/30
3/3 [==============================] - 68s 20s/step - loss: 0.6211 - accuracy: 0.6944 - precision: 0.8636 - recall: 0.5000 - val_loss: 0.6394 - val_accuracy: 0.5882 - val_precision: 1.0000 - val_recall: 0.3000
Epoch 3/30
3/3 [==============================] - 67s 20s/step - loss: 0.5208 - accuracy: 0.7639 - precision: 0.9200 - recall: 0.6053 - val_loss: 0.4496 - val_accuracy: 0.8235 - val_precision: 0.8889 - val_recall: 0.8000
Epoch 4/30
3/3 [==============================] - 66s 20s/step - loss: 0.4399 - accuracy: 0.7917 - precision: 0.7255 - recall: 0.9737 - val_loss: 0.4152 - val_accuracy: 0.8824 - val_precision: 0.9000 - val_recall: 0.9000
Epoch 5/30
3/3 [==============================] - 64s 19s/step - loss: 0.4365 - accuracy: 0.8056 - precision: 0.730

IndexError: pop from empty list

In [10]:
# print kfold results
fold_loss=np.array(fold_loss)
fold_accuracy=np.array(fold_accuracy)
fold_precision=np.array(fold_precision)
fold_recall=np.array(fold_recall)

print(f'Average Loss: {np.mean(fold_loss)}')
print(f'Average Accuracy: {np.mean(fold_accuracy)} (+/- {np.std(fold_accuracy)})')
print(f'Average Precision: {np.mean(fold_precision)} (+/- {np.std(fold_precision)})')
print(f'Average Recall: {np.mean(fold_recall)} (+/- {np.std(fold_recall)})')

Average Loss: 0.2781157370656729
Average Accuracy: 0.8758170008659363 (+/- 0.07497845513720999)
Average Precision: 0.8711111187934876 (+/- 0.07875575151321046)
Average Recall: 0.8933333396911621 (+/- 0.09469899770066628)


In [11]:
# load the test examples and labels
text_files_test={}
load_files('/kaggle/input/patient-text-test-files/patient_txt_test_files', text_files_test)
text_files_test = collections.OrderedDict(sorted(text_files_test.items()))

X_test=[]
for key in text_files_test:
    X_test.append(text_files_test[key])
X_test=np.array(X_test)

y_test=pd.read_csv('/kaggle/input/solution/Solution.csv')
y_test.sort_values(y_test.columns[0], axis=0, inplace=True)
y_test=y_test['Label']
y_test.loc[y_test > 0] = 1
y_test=np.asarray(y_test).astype('float32')

X_train=X
y_train=y

In [12]:
# train and evaluate the model on the full training and testing dataset
with tpu_strategy.scope():
    create_run_model(X_train, X_test, y_train, y_test, fold_loss, fold_accuracy, fold_precision, fold_recall, 1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/30


2024-09-24 02:58:28.564190: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


3/3 [==============================] - 67s 6s/step - loss: 0.7243 - accuracy: 0.4944 - precision: 0.5231 - recall: 0.7083 - val_loss: 0.5618 - val_accuracy: 0.8333 - val_precision: 0.7647 - val_recall: 1.0000
Epoch 2/30
3/3 [==============================] - 1s 673ms/step - loss: 0.5338 - accuracy: 0.7640 - precision: 0.7015 - recall: 0.9792 - val_loss: 0.4720 - val_accuracy: 0.9167 - val_precision: 0.8667 - val_recall: 1.0000
Epoch 3/30
3/3 [==============================] - 1s 678ms/step - loss: 0.4994 - accuracy: 0.7865 - precision: 0.7377 - recall: 0.9375 - val_loss: 0.4007 - val_accuracy: 0.8750 - val_precision: 0.8571 - val_recall: 0.9231
Epoch 4/30
3/3 [==============================] - 1s 647ms/step - loss: 0.3632 - accuracy: 0.8652 - precision: 0.8333 - recall: 0.9375 - val_loss: 0.3594 - val_accuracy: 0.8750 - val_precision: 0.8571 - val_recall: 0.9231
Epoch 5/30
3/3 [==============================] - 1s 655ms/step - loss: 0.3467 - accuracy: 0.8876 - precision: 0.8519 - recal